<a href="https://colab.research.google.com/github/MINAADELMARKOS/TheAvengersAnalysis/blob/main/Data_Analysis_using_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Pyspark**

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9fbd855cb2615398a46694eaff2affee8b12f88be8504b8c524c24653982b5be
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql.types import Row
from datetime import datetime
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

In [ ]:
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
spark = SparkSession(sc)
SQLContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
movies_record = sc.parallelize([
                                Row(certificate_id = 2195194,
                                    movie_name = "Iron Man",
                                    hit = True,
                                    category = ['action','thriller'],
                                    rating = {"IMDB":7.9 , "rotten tomatoes" : 7.7},
                                    release_time = datetime(2008,5,1,13,1,5)
                                    ),

                                 Row(certificate_id = 2195204,
                                    movie_name = "Baywastch",
                                    hit = False,
                                    category = ['comedy','action'],
                                    rating = {"IMDB":5.6 , "rotten tomatoes" :4.0},
                                    release_time = datetime(2017,5,12,3,2,5)
                                    )
])

In [ ]:
movies_record_df = movies_record.toDF()
movies_record_df.show()

+--------------+----------+-----+------------------+--------------------+-------------------+
|certificate_id|movie_name|  hit|          category|              rating|       release_time|
+--------------+----------+-----+------------------+--------------------+-------------------+
|       2195194|  Iron Man| true|[action, thriller]|{IMDB -> 7.9, rot...|2008-05-01 13:01:05|
|       2195204| Baywastch|false|  [comedy, action]|{IMDB -> 5.6, rot...|2017-05-12 03:02:05|
+--------------+----------+-----+------------------+--------------------+-------------------+



**Temporary view valid for just this session**

In [ ]:
movies_record_df.createOrReplaceTempView('records')

In [ ]:
all_movie_records_df = SQLContext.sql("select * from records")

all_movie_records_df.show()

+--------------+----------+-----+------------------+--------------------+-------------------+
|certificate_id|movie_name|  hit|          category|              rating|       release_time|
+--------------+----------+-----+------------------+--------------------+-------------------+
|       2195194|  Iron Man| true|[action, thriller]|{IMDB -> 7.9, rot...|2008-05-01 13:01:05|
|       2195204| Baywastch|false|  [comedy, action]|{IMDB -> 5.6, rot...|2017-05-12 03:02:05|
+--------------+----------+-----+------------------+--------------------+-------------------+



In [ ]:
SQLContext.sql('select certificate_id,category[1], rating["IMDB"] from records').show()

+--------------+-----------+------------+
|certificate_id|category[1]|rating[IMDB]|
+--------------+-----------+------------+
|       2195194|   thriller|         7.9|
|       2195204|     action|         5.6|
+--------------+-----------+------------+



In [ ]:
SQLContext.sql('select certificate_id, NOT hit from records').show()

+--------------+---------+
|certificate_id|(NOT hit)|
+--------------+---------+
|       2195194|    false|
|       2195204|     true|
+--------------+---------+



In [ ]:
SQLContext.sql("select * from records where hit = 'false' ").show()

+--------------+----------+-----+----------------+--------------------+-------------------+
|certificate_id|movie_name|  hit|        category|              rating|       release_time|
+--------------+----------+-----+----------------+--------------------+-------------------+
|       2195204| Baywastch|false|[comedy, action]|{IMDB -> 5.6, rot...|2017-05-12 03:02:05|
+--------------+----------+-----+----------------+--------------------+-------------------+



In [ ]:
SQLContext.sql("select * from records where rating['IMDB'] < 6.0 ").show()

+--------------+----------+-----+----------------+--------------------+-------------------+
|certificate_id|movie_name|  hit|        category|              rating|       release_time|
+--------------+----------+-----+----------------+--------------------+-------------------+
|       2195204| Baywastch|false|[comedy, action]|{IMDB -> 5.6, rot...|2017-05-12 03:02:05|
+--------------+----------+-----+----------------+--------------------+-------------------+



In [ ]:
SQLContext.sql("select * from records where release_time >= \'2010-05-01 0:0:0\'").show()

+--------------+----------+-----+----------------+--------------------+-------------------+
|certificate_id|movie_name|  hit|        category|              rating|       release_time|
+--------------+----------+-----+----------------+--------------------+-------------------+
|       2195204| Baywastch|false|[comedy, action]|{IMDB -> 5.6, rot...|2017-05-12 03:02:05|
+--------------+----------+-----+----------------+--------------------+-------------------+



# **Temporary view Shared across multiple sessions**

In [ ]:
movies_record_df.createGlobalTempView("global_records")

In [ ]:
SQLContext.sql('select * from global_temp.global_records').show()

+--------------+----------+-----+------------------+--------------------+-------------------+
|certificate_id|movie_name|  hit|          category|              rating|       release_time|
+--------------+----------+-----+------------------+--------------------+-------------------+
|       2195194|  Iron Man| true|[action, thriller]|{IMDB -> 7.9, rot...|2008-05-01 13:01:05|
|       2195204| Baywastch|false|  [comedy, action]|{IMDB -> 5.6, rot...|2017-05-12 03:02:05|
+--------------+----------+-----+------------------+--------------------+-------------------+

